In [30]:
import time
import os
import math
import cv2
import numpy as np
import pandas as pd
import pyautogui as pa
import shutil
import imageio
from PIL import Image, ImageDraw
import random
from scipy.stats import norm, kstest, sem, ttest_ind, linregress
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from lib.analysis.cell_roi import RandomROISample
from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.TIF import *
from lib.analysis.align import ImageAlign
from lib.utilities import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.analysis.barrel_roi import Barrel_ROI_Creator
from lib.analysis.false_discovery_control import false_discovery_control
from lib.analysis.trace_metrics import TraceMetrics

from lib.analysis.movie_maker import MovieMaker
###########################################
# Now we allow 3 barrels in a field of view to be analyzed
# in catalog, number of barrels is saved in Num_Barrels column
# The <slic>_<loc>_barrel_boundary.dat file is in ascending order of barrels, one
#   separate ROI for each barrel
###########################################

In [31]:
date, slic, loc, rec =  "12-20-24", 1, 3, 1
date, slic, loc, rec =  "10-29-24", 1, 1, 3
date, slic, loc, rec =  "12-09-24", 1, 1, 1
date, slic, loc, rec, zda_subdir, subdir_2 =  "10-23-24", 1, 2, 3, "selected_zda", 'L23_vs_L4_NBQX/L4_NBQX_2'
date, slic, loc, rec, zda_subdir, subdir_2 =  "12-09-24", 2, 1, 1, "converted_zda", 'L23_vs_L4_NBQX/L4_NBQX'
data_dir = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/" + date + f"/{zda_subdir}/analysis0{slic}_0{loc}_0{rec}/"
trace_file = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/{date}/{zda_subdir}/_" + f"{slic}_{loc}_{rec}_trace_{slic}_rois.dat"
trace_supra_file = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/{date}/{zda_subdir}/" + f"supragranular_{slic}_{loc}_{rec}_trace_{slic}_rois_supragranular.dat"
ladder_df = 'C:/Users/jjudge3/Desktop/Data/crossing_PPR/export_summary_final_velocity_ladders_backup.csv'
ladder_traces = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/" + date + f"/{zda_subdir}/ladder_2_1_1_trace_2_rois_output__ladder_.dat"
ladder_df = pd.read_csv(ladder_df, sep=',', header=0)
start_frame = 97
end_frame = 120
frame_step_size = 1
print(data_dir)
def read_dat_trace_file(filename):
    data_arr = pd.read_csv(filename,
                                 sep='\t',
                                 header=0)
    return data_arr
# strip zeros from date day and month
formatted_date = pd.to_datetime(date).strftime('%m/%d/%Y').lstrip("0").replace("/0", "/")
print(formatted_date)
rec_ladder_df = ladder_df[(ladder_df['Date'] == formatted_date) & (ladder_df['Slice'] == slic) & (ladder_df['Location'] == loc) & (ladder_df['Recording'] == rec)]
rec_ladder_df

C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/
12/9/2024


,ROI,amp,snr,latency,halfwidth,Stim_Distance,X_Center,Y_Center,ROI_Set,Date,...,Coronal/Sagittal,IPI,Is_Pulse_2,Dist_from_home_barrel,resides_in_barrel,instantaneous_velocity,ROI file,Medial/Lateral,Age,Sex
344,1,0.859922,10.19070,3.5000,22.39960,236.448726,60.241379,13.172414,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,507.399931,NaN,34.576583,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
345,2,1.411660,11.11860,2.9987,28.78030,224.659743,58.955056,15.898876,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,490.066690,NaN,94.065929,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
346,3,1.927120,10.42950,2.8120,28.40410,204.792578,57.511111,18.544444,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,472.504581,Neighbor,NaN,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
347,4,2.286390,9.91073,2.8113,26.47860,183.172050,55.285714,20.659341,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,454.125955,NaN,237.197565,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
348,5,2.530260,13.64760,2.8843,23.89080,174.413302,53.958333,23.364583,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,436.810533,Neighbor,782.988696,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
349,6,2.700910,20.10300,2.9074,18.17140,152.970585,52.100000,25.755556,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,418.723494,Neighbor,64.788278,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
350,7,2.954290,21.77690,2.6332,15.15830,146.109548,50.333333,28.161616,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,400.958548,Neighbor,83.585081,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
351,8,3.285020,43.76620,2.4223,13.60300,131.041978,48.731183,30.698925,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,383.330454,NaN,49.404910,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
352,9,5.263260,21.86550,2.0577,10.98350,121.342490,46.885417,33.093750,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,365.317424,NaN,72.662501,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F
353,10,8.679340,18.49250,1.8143,10.79110,108.000000,45.093750,35.468750,2_rois_output__ladder_,12/9/2024,...,NaN,NaN,False,347.631371,Home,83.774637,C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23...,Lateral,79.0,F


In [39]:
trace_data = read_dat_trace_file(trace_file)
time = trace_data['Pt'].values * 0.5

trace_data_supra = read_dat_trace_file(trace_supra_file)

# vmax is max across all columns except 'Pt'
vmax = trace_data.drop(columns=['Pt']).max().max()
vmin = trace_data.drop(columns=['Pt']).min().min()

vmax_supra = trace_data_supra.drop(columns=['Pt']).max().max() - 1
vmin_supra = trace_data_supra.drop(columns=['Pt']).min().min() - 1

trace_data_ladder = read_dat_trace_file(ladder_traces)
vmin = min(vmin, vmin_supra)
stim_time = 96.6
trace_data_ladder


,Pt,ROI1,ROI2,ROI3,ROI4,ROI5,ROI6,ROI7,ROI8,ROI9,...,ROI20,ROI21,ROI22,ROI23,ROI24,ROI25,ROI26,ROI27,ROI28,ROI29
0,0,-0.072874,-0.099102,0.113619,0.211879,0.102169,0.088255,0.014100,-0.051434,-0.022491,...,0.499866,0.553469,0.447628,0.258259,0.285886,0.261635,-0.127896,-0.029927,0.180614,0.577156
1,1,-0.110530,-0.114534,0.074312,0.140251,0.053905,0.052218,-0.031353,-0.087840,0.011011,...,0.457597,0.514928,0.412572,0.193876,0.170436,0.203667,-0.176039,-0.068179,0.187810,0.434192
2,2,-0.122305,-0.098993,0.061338,0.087371,0.035352,0.041419,-0.054731,-0.097711,0.078054,...,0.446769,0.489694,0.385768,0.160807,0.089276,0.156839,-0.209932,-0.114338,0.181141,0.284882
3,3,-0.107618,-0.063088,0.074556,0.064390,0.041150,0.044532,-0.053544,-0.085858,0.156752,...,0.466148,0.472052,0.357981,0.151655,0.058909,0.122245,-0.232572,-0.187502,0.138897,0.164091
4,4,-0.083759,-0.033880,0.095239,0.083377,0.060612,0.054311,-0.022940,-0.058302,0.221583,...,0.494550,0.449335,0.319507,0.136517,0.063591,0.092275,-0.263306,-0.303336,0.039750,0.086156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,894,-0.218343,-0.059387,-0.044685,-0.101988,-0.057628,-0.152157,-0.303732,-0.444509,-0.317500,...,-0.166626,-0.118006,-0.227345,-0.218751,-0.268038,-0.386386,-0.446354,-0.458967,-0.252808,-0.128910
895,895,-0.241823,-0.072619,-0.078255,-0.118314,-0.106859,-0.165404,-0.338000,-0.482298,-0.337644,...,-0.183588,-0.169330,-0.246909,-0.266158,-0.327524,-0.446962,-0.558301,-0.582733,-0.414238,-0.237150
896,896,-0.277442,-0.111081,-0.091171,-0.133341,-0.140284,-0.157008,-0.340038,-0.489656,-0.311458,...,-0.192776,-0.224029,-0.282805,-0.318717,-0.367583,-0.502550,-0.653755,-0.675741,-0.521052,-0.320420
897,897,-0.332561,-0.185029,-0.093600,-0.150627,-0.141367,-0.130930,-0.315692,-0.478487,-0.253254,...,-0.222908,-0.272701,-0.339461,-0.383234,-0.403160,-0.543449,-0.725948,-0.729598,-0.550420,-0.377209


In [74]:
# create a movie of a png image next to a trace plot
custom_frames = []
frame_ratio = 1
plot_type = 'trace-vert'  # trace
min_lat = np.min(rec_ladder_df['latency'])
for frame in range(start_frame, end_frame, frame_step_size):
    for i_partial in range(frame_ratio):
        fig, ax = plt.subplots(1, 2, figsize=(10, 6))
        image_path = os.path.join(data_dir, f"{frame}.jpg")
        print(f"Processing image {frame}: {image_path}")
        
        # display the image with matplotlib
        if frame < end_frame + 1:
            img = plt.imread(image_path)
        ax[0].imshow(img)
        ax[0].axis('off')  # Hide axes

        # right subplot for trace plot
        if plot_type == 'trace':
            for col in trace_data:
                if col != 'Pt':
                    ax[1].plot(time[:frame], trace_data[col][:frame], label=col)
            for col in trace_data_supra:
                if col != 'Pt':
                    ax[1].plot(time[:frame], trace_data_supra[col][:frame] - 1, label=col)
            ax[1].set_xlim(25, time[end_frame + 30])
            ax[1].set_ylim(vmin, vmax)
            ax[1].set_xlabel('Time (ms)')
            ax[1].set_ylabel('ΔF/F')
            ax[1].set_yticks([])
            
            # mark stim time with a vertical line
            ax[1].axvline(x=stim_time * 0.5, color='b', linestyle='--')
            x_ticks = [i + stim_time * 0.5 for i in [-20, -10, 0, 10, 20, 30, 40]]
            print(x_ticks)
            ax[1].set_xticks(x_ticks)
            ax[1].set_xticklabels([i for i in range(-20, 41, 10)])
        elif plot_type == 'latency':
            # iterate through latencies and plot those whose time is less than the current frame
            num_pts_plotted = 0

            for index, row in rec_ladder_df.iterrows():
                lat = row['latency']
                
                if lat > 10:
                    continue
                elif lat * 2 + stim_time < float(frame) + float(i_partial) / float(frame_ratio):
                    dist = row['Dist_from_home_barrel']
                    # star marker
                    ax[1].scatter(dist, lat + 1, 
                                  color='tab:blue' if (lat < 1.8 or abs(dist-300) < 90) else 'tab:green' if lat < 2.1 else ('tab:orange' if dist > 300 else 'tab:green'),
                                  marker='*' if lat <= min_lat else 'o')
                    num_pts_plotted += 1
            print('num_pts_plotted:', num_pts_plotted)

            ax[1].set_xlim(50, 550)
            ax[1].set_ylim(0, 6)
            ax[1].set_xlabel('Distance along L4 (μm)')
            ax[1].set_ylabel('Latency (ms)')
            ax[1].set_xticks([100, 200, 300, 400, 500])
            ax[1].set_xticklabels([-200, -100, 0, 100, 200])

            # mark the current time with a horizontal dotted line
            ax[1].axhline(y=(float(frame) + float(i_partial) / float(frame_ratio) - stim_time) * 0.5 + 1, color='black', linestyle='--')
            if i_partial == 0:
                plt.show()

        elif plot_type == 'trace-vert':
            # iterate through latencies and plot those whose time is less than the current frame
            num_pts_plotted = 0

            # plot all traces vertically at the position of their corresponding distance
            for col in trace_data_ladder:
                if col == 'Pt':
                    continue
                if col.startswith('ROI'):
                    roi_no = int(col[3:])
                
                # look up ROI num in rec_ladder_df
                dist = rec_ladder_df[rec_ladder_df['ROI'] == roi_no]['Dist_from_home_barrel'].values[0]
                lat = rec_ladder_df[rec_ladder_df['ROI'] == roi_no]['latency'].values[0]

                # take only points [stim_time, stim_time + 12] from trace_data
                trace_pts = trace_data_ladder[(trace_data_ladder['Pt'] < frame + i_partial / frame_ratio)]

                # plot the trace vertically at the position of their corresponding distance, connect points in scatter 
                ax[1].plot( trace_pts[col] * 10 + dist, (trace_pts['Pt'] - stim_time + 10) * 0.5, 
                           color='tab:blue' if (lat < 1.8 or abs(dist-300) < 90) else 'tab:green' if lat < 2.1 else ('tab:orange' if dist > 300 else 'tab:green'), 
                           alpha=0.5)


            ax[1].set_xlim(50, 550)
            ax[1].set_ylim(0, 20)
            ax[1].set_xlabel('Distance along L4 (μm)')
            ax[1].set_ylabel('Time (ms)')
            ax[1].set_xticks([100, 200, 300, 400, 500])
            ax[1].set_xticklabels([-200, -100, 0, 100, 200])

        # save the figure to file
        output_path = os.path.join(data_dir, f"custom_frame_{frame}_{i_partial}.png")
        fig.savefig(output_path, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)  # Close the figure to free memory
        custom_frames.append(output_path)

# create a movie from the saved frames
custom_frames = [imageio.imread(frame) for frame in custom_frames]
imageio.mimsave(os.path.join(data_dir, 'custom_movie velocity v6.mp4'), custom_frames, fps=10)
print(f"Custom movie saved to {data_dir}custom_movie.mp4")
        


Processing image 97: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/97.jpg
Processing image 98: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/98.jpg
Processing image 99: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/99.jpg
Processing image 100: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/100.jpg
Processing image 101: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/101.jpg
Processing image 102: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/102.jpg
Processing image 103: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/103.jpg
Processing image 104: C:/Users/jjudge3/Desktop/Data/cr

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (794, 530) to (800, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Custom movie saved to C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/custom_movie.mp4


In [68]:
# create a movie from the saved frames
#custom_frames = [imageio.imread(frame) for frame in custom_frames]
imageio.mimsave(os.path.join(data_dir, 'custom_movie v5.mp4'), custom_frames, fps=10)
print(f"Custom movie saved to {data_dir}custom_movie.mp4")

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1149, 530) to (1152, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Custom movie saved to C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX/12-09-24/converted_zda/analysis02_01_01/custom_movie.mp4
